In [1]:
import pandas as pd

# Memuat dataset yang akan digunakan
dataset = pd.read_csv('./data/politik_merge.csv')

# Overview dari dataset
dataset.head()

,Judul,Waktu,Link,Content,tag1,tag2,tag3,tag4,tag5,source
0,Jokowi Kenakan Pakaian Adat Betawi di Sidang T...,16/08/2024,https://nasional.kompas.com/read/2024/08/16/11...,"JAKARTA, KOMPAS.com - Presiden Joko Widodo me...",Presiden Jokowi,Jokowi,sidang tahunan MPR RI 2024,Jokowi adat Betawi sidang mpr 2024,Megawati tak hadiri sidang tahunan MPR 2024,kompas
1,Amnesty International Beberkan 6 Indikator Kri...,2024-07-18,https://nasional.tempo.co/read/1893144/amnesty...,"TEMPO.CO, Jakarta - Amnesty International Indo...",Amnesty International,Amnesty International Indonesia,Kebebasan Berpendapat,Indeks Demokrasi,Revisi UU TNI,tempo
2,"Jelang Long Weekend, Stasiun Kereta Cepat Hali...","Rabu, 08 Mei 2024 19:18 WIB",https://news.detik.com/berita/d-7331666/jelang...,"Stasiun kereta cepat Whoosh di Halim, Jakarta ...",kereta cepat whoosh,stasiun halim,long weekend,NaN,NaN,detik
3,KPU Tegaskan Pemilih Tak Terdaftar di DPT Bisa...,13/02/2024,https://nasional.kompas.com/read/2024/02/13/21...,"JAKARTA, KOMPAS.com - Komisi Pemilihan Umum (...",KPU,pemilu 2024,Hasyim Asy'ari,NaN,NaN,kompas
4,Kemenag Luncurkan Gerakan Senam Haji Jaga Keta...,2024-04-29,https://nasional.tempo.co/read/1861810/kemenag...,"TEMPO.CO, Jakarta - Kementerian Agama atau Kem...",Senam Haji,Kemenag,Jemaah Haji,Ibadah Haji,Asrama Haji,tempo


In [2]:
# Jumlah baris dan kolom dalam dataset
dataset.shape

(45781, 10)

In [3]:
# Informasi lebih lanjut tentang dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45781 entries, 0 to 45780
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Judul    45746 non-null  object
 1   Waktu    45781 non-null  object
 2   Link     45779 non-null  object
 3   Content  45742 non-null  object
 4   tag1     45703 non-null  object
 5   tag2     45550 non-null  object
 6   tag3     41213 non-null  object
 7   tag4     31040 non-null  object
 8   tag5     19892 non-null  object
 9   source   45709 non-null  object
dtypes: object(10)
memory usage: 3.5+ MB


In [4]:
# Mengubah nama kolom untuk konsistensi
dataset.columns = dataset.columns.str.lower().str.replace(' ', '_')
dataset.head(1)

,judul,waktu,link,content,tag1,tag2,tag3,tag4,tag5,source
0,Jokowi Kenakan Pakaian Adat Betawi di Sidang T...,16/08/2024,https://nasional.kompas.com/read/2024/08/16/11...,"JAKARTA, KOMPAS.com - Presiden Joko Widodo me...",Presiden Jokowi,Jokowi,sidang tahunan MPR RI 2024,Jokowi adat Betawi sidang mpr 2024,Megawati tak hadiri sidang tahunan MPR 2024,kompas


In [5]:
# Menghapus kolom yang tidak diperlukan
dataset = dataset.drop(columns=['judul', 'waktu', 'link', 'tag1', 'tag2', 'tag3', 'tag4', 'tag5', 'source'])
dataset.head(1)

,content
0,"JAKARTA, KOMPAS.com - Presiden Joko Widodo me..."


In [6]:
# Menghitung jumlah nilai yang hilang (missing values) dalam dataset
dataset.isna().sum()

content    39
dtype: int64

In [7]:
# Rasio dari nilai yang hilang terhadap total data
missing_ratio = dataset.isna().sum() / len(dataset)
print(f"Rasio nilai yang hilang: {missing_ratio}")

Rasio nilai yang hilang: content    0.000852
dtype: float64


In [8]:
# Menhapus baris yang memiliki nilai yang hilang
dataset = dataset.dropna()
dataset.isna().sum()

content    0
dtype: int64

In [9]:
import re

# Preprocessing teks untuk membersihkan konten
def clean_text(text):
    """Preprocessing teks yang akurat dan efisien"""
    if not isinstance(text, str):
        return ""
    
    # Hapus URL
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # Hapus mention
    text = re.sub(r'@\w+', '', text)
    
    # Ubah hashtag jadi kata biasa
    text = re.sub(r'#(\w+)', r'\1', text)
    
    # Hapus karakter khusus kecuali apostrof dan tanda hubung
    text = re.sub(r'[^\w\s\'\-]', ' ', text)
    
    # Normalisasi apostrof
    text = re.sub(r'[''`]', "'", text)
    
    # Hapus angka standalone
    text = re.sub(r'\b\d+\b', '', text)
    
    # Hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Hapus kata pendek kecuali yang penting
    words = text.split()
    important_short = {'di', 'ke', 'ku', 'mu', 'ya', 'si', 'se', 'dan', 'ini', 'itu', 'dia', 'ada', 'tak', 'pun', 'nya', 'aku', 'bab', 'hal', 'per', 'pro', 'sub', 'non', 'bio', 'dll', 'dsb', 'pdt', 'dr', 'ir', 'st', 'mt', 'sh', 'mm', 'sm', 'sk', 'sd', 'tk', 'rs', 'rt', 'rw', 'no'}
    filtered_words = [word for word in words if len(word) >= 2 or word in important_short]
    
    return ' '.join(filtered_words)

# Terapkan ke dataset
dataset['cleaned_content'] = dataset['content'].apply(clean_text)
print(dataset['cleaned_content'].sample(1).values[0])

TEMPO CO Cirebon Dampak one way yang diberlakukan mulai KM Gerbang Tol Kalikangkung hingga KM Tol Cipali membuat jalur pantura mengalami kepadatan pada Sabtu April Seperti diketahui one way di jalur tol mulai diberlakukan hari ini Sabtu April mulai pukul WIB One way yang diberlakukan mulai KM GT Kalikangkung hingga KM tol Cipali Berdasarkan pantauan jalur pantura arah Cirebon menuju Jawa Tengah padat oleh kendaraan roda empat sedangkan di jalur sebaliknya dipenuhi oleh pemudik sepeda motor yang semakin banyak jumlahnya dan hendak kembali ke Jakarta Arus Balik Libur Idul Adha Ribu Kendaraan Telah Kembali ke Jabodetabek Untuk menghindari kepadatan kendaraan yang hendak menuju Jawa Tengah dibagi dua di Kedawung Ada yang diluruskan menuju by pass di jalur pantura dan ada pula yang dibelokkan ke kiri masuk ke Pilang-Kota Cirebon-Samadikun Yos Sudarso dan kembali ke jalur Pantura Untuk pemudik di jalur Pantura dengan tujuan Jakarta dan sekitarnya hingga pukul WIB jumlahnya sudah mencapai ken

In [10]:
# Case folding untuk normalisasi huruf kecil
dataset['cleaned_content'] = dataset['cleaned_content'].str.lower()
print(dataset['cleaned_content'].sample(1).values[0])

tempo co jakarta komisi pemberantasan korupsi atau kpk menangkap bupati labuhanbatu erik atrada ritonga ear dalam operasi tangkap tangan atau ott benar salah satunya bupati labuhanbatu kata juru bicara penindakan dan kepegawaian kpk ali fikri kamis januari erik ialah seorang dokter yang kemudian menjabat sebagai bupati labuhanbatu periode setelah menjadi anggota komisi xi dpr-ri dari fraksi partai hanura dapil sumut periode ia maju di pilkada bupati labuhanbatu bersama pasangannya ellya rosa siregar saat ahmad sahroni bilang pelanggaran etik nurul ghufron jadi catatan seleksi capim kpk pada mei kpu kabupaten labuhanbatu resmi menetapkan pasangan calon erik-ellya sebagai bupati dan wakil bupati terpilih labuhanbatu periode erik lahir di rantau prapat labuhanbatu sumatera utara pada mei pria berusia tahun itu mengeyam pendidikan sd dan smp di rantau prapat kemudian saat melanjutkan ke sma ia bersekolah di sma negeri bandung ia melanjutkan studi s1 di universitas sumatera utara dan s2 di 

In [11]:
from nltk.tokenize import word_tokenize

# Proses tokenisasi 
dataset['tokens'] = [word_tokenize(text) for text in dataset['cleaned_content']]
print(dataset['tokens'].sample(1).values[0])

['jakarta', 'kompas', 'com', 'sekretaris', 'jenderal', 'sekjen', 'partai', 'amanat', 'nasional', 'pan', 'eddy', 'soeparno', 'menegaskan', 'melawan', 'kotak', 'kosong', 'dalam', 'sebuah', 'kontestasi', 'pilkada', 'bukanlah', 'hal', 'yang', 'mudah', 'menurut', 'dia', 'pernah', 'ada', 'calon', 'kepala', 'daerah', 'yang', 'kalah', 'ketika', 'melawan', 'kotak', 'kosong', 'di', 'pilkada', 'jangan', 'lupa', 'loh', 'yang', 'namanya', 'melawan', 'kotak', 'kosong', 'itu', 'juga', 'enggak', 'gampang', 'ada', 'kejadian', 'di', 'mana', 'kotak', 'kosong', 'itu', 'bisa', 'menang', 'pilkada', 'ujar', 'eddy', 'saat', 'ditemui', 'di', 'gedung', 'dpr', 'senayan', 'jakarta', 'rabu', 'terkait', 'skenario', 'koalisi', 'indonesia', 'maju', 'kim', 'plus', 'menjegal', 'anies', 'baswedan', 'di', 'pilkada', 'jakarta', 'eddy', 'membantah', 'kecurigaan', 'tersebut', 'dia', 'mengeklaim', 'dua', 'pasangan', 'calon', 'bisa', 'tercipta', 'di', 'pilkada', 'jakarta', 'saya', 'masih', 'coba', 'memahami', 'bagaimana', 'ca

In [12]:
from nltk.corpus import stopwords

# Proses stopword removal
stop_words_frozen = frozenset(stopwords.words('indonesian'))
dataset['stopwords'] = [
    [word for word in tokens if word not in stop_words_frozen]
    for tokens in dataset['tokens']
]

print(dataset['stopwords'].sample(1).values[0])

['komisi', 'pemilihan', 'kpu', 'ri', 'menetapkan', 'calon', 'anggota', 'legislatif', 'caleg', 'dpr', 'ri', 'terpilih', 'periode', 'politikus', 'pdip', 'said', 'abdullah', 'caleg', 'terpilih', 'perolehan', 'suara', 'tertinggi', 'suara', 'penetapan', 'dibacakan', 'ketua', 'kpu', 'ri', 'mochammad', 'afifuddin', 'kantor', 'kpu', 'menteng', 'jakarta', 'pusat', 'minggu', 'penetapan', 'dihadiri', 'mahkamah', 'konstitusi', 'mk', 'bawaslu', 'dkpp', 'partai', 'politik', 'stakeholder', 'terkait', 'penetapan', 'said', 'abdullah', 'caleg', 'terpilih', 'perolehan', 'suara', 'tertinggi', 'suara', 'posisi', 'dedi', 'mulyadi', 'partai', 'gerindra', 'suara', 'urutan', 'ketiga', 'edhie', 'baskoro', 'yudhoyono', 'ibas', 'partai', 'demokrat', 'suara', 'puan', 'maharani', 'posisi', 'keenam', 'suara', 'rincian', 'caleg', 'dpr', 'ri', 'terpilih', 'suara', 'tertinggi', 'said', 'abdullah', 'pdip', 'suara', 'dedi', 'mulyadi', 'gerindra', 'suara', 'edhie', 'baskoro', 'yudhoyono', 'partai', 'demokrat', 'suara', 'h

In [13]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re

# Inisialisasi stemmer sekali saja
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def clean_token(token):
    """Bersihkan token sebelum stemming"""
    if not token or not isinstance(token, str):
        return token
    
    cleaned = re.sub(r'[^a-zA-Z]', '', token.lower())
    return cleaned if cleaned else token

def build_accurate_stem_dict(df):
    """Build stemming dictionary dengan akurasi maksimal"""
    print("Collecting unique words...")
    all_words = set()
    
    # Kumpulkan semua kata unik
    for tokens in df['stopwords']:
        if tokens and isinstance(tokens, list):
            for token in tokens:
                if token and isinstance(token, str) and len(token.strip()) > 0:
                    all_words.add(token.strip())
    
    print(f"Total unique words: {len(all_words):,}")
    
    # Stem setiap kata individual
    stem_dict = {}
    processed = 0
    errors = 0
    
    for word in all_words:
        try:
            cleaned_word = clean_token(word)
            
            if cleaned_word and len(cleaned_word) > 1:
                stemmed = stemmer.stem(cleaned_word)
                
                if stemmed and len(stemmed) <= len(cleaned_word) + 2:
                    stem_dict[word] = stemmed
                else:
                    stem_dict[word] = cleaned_word
                    errors += 1
            else:
                stem_dict[word] = word
                
        except Exception:
            stem_dict[word] = word
            errors += 1
        
        processed += 1
        if processed % 5000 == 0:
            print(f"Processed {processed:,}/{len(all_words):,}")
    
    print(f"Completed! Errors: {errors}")
    return stem_dict

# Build dictionary dengan akurasi maksimal
stem_dict = build_accurate_stem_dict(dataset)

# Apply stemming
dataset['stemmed'] = [[stem_dict.get(token, token) for token in tokens] for tokens in dataset['stopwords']]

# Lihat hasil
print("Sample result:")
print(dataset['stemmed'].sample(1).values[0])

Total unique words: 122,896
Processed 5,000/122,896
Processed 10,000/122,896
Processed 15,000/122,896
Processed 20,000/122,896
Processed 25,000/122,896
Processed 30,000/122,896
Processed 35,000/122,896
Processed 40,000/122,896
Processed 45,000/122,896
Processed 50,000/122,896
Processed 55,000/122,896
Processed 60,000/122,896
Processed 65,000/122,896
Processed 70,000/122,896
Processed 75,000/122,896
Processed 80,000/122,896
Processed 85,000/122,896
Processed 90,000/122,896
Processed 95,000/122,896
Processed 100,000/122,896
Processed 105,000/122,896
Processed 110,000/122,896
Processed 115,000/122,896
Processed 120,000/122,896
Completed! Errors: 20
Sample result:
['jakarta', 'kompas', 'com', 'wakil', 'ketua', 'badan', 'legislasi', 'baleg', 'dpr', 'achmad', 'baidowi', 'dpr', 'batal', 'bahas', 'revisi', 'undangundang', 'tni', 'undangundang', 'polri', 'awiek', 'sapa', 'akrab', 'revisi', 'uu', 'bahas', 'dpr', 'kunjung', 'terima', 'surat', 'presiden', 'surpres', 'revisi', 'uu', 'tni', 'uu', 'p

In [14]:
# Menampilkan hasil akhir
dataset.head(1)

,content,cleaned_content,tokens,stopwords,stemmed
0,"JAKARTA, KOMPAS.com - Presiden Joko Widodo me...",jakarta kompas com presiden joko widodo memaka...,"[jakarta, kompas, com, presiden, joko, widodo,...","[jakarta, kompas, com, presiden, joko, widodo,...","[jakarta, kompas, com, presiden, joko, widodo,..."


In [15]:
# Simpan dataset yang sudah dibersihkan
dataset.to_csv('data/preprocessed_dataset.csv', index=False)